In [1]:
import requests
import os
import re
import json
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
import time
import matplotlib.pyplot as plt
import urllib
%matplotlib inline
from datetime import date

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time


In [2]:

# Setup the ChromeDriver with a valid User-Agent
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Optional, to run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


# Set a valid User-Agent header
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(options=options)

headers = {
    'Accept-Language': 'en',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'X-FORWARDED-FOR': '2.21.184.0'
}

In [36]:
pmg = 'https://pmg.org.za/question_replies/'

In [37]:
response = requests.get(pmg, headers=headers)
print(response.status_code)

200


In [38]:
pmg_soup = BeautifulSoup(response.text, 'html.parser')

In [39]:
def safe_extract(soup, selector, attribute, default=np.nan):
        """Helper function to extract text from a BeautifulSoup tag safely."""
        element = soup.find(selector, attribute)
        return element.get_text(strip=True) if element else default


# Helper function to extract text from multiple <p> tags within a specific div
def safe_extract_multiple(soup, parent_selector, parent_attribute, child_selector='p'):
    """Helper function to extract text from multiple <p> tags safely inside a parent container."""
    parent_element = soup.find(parent_selector, parent_attribute)
    if parent_element:
        child_elements = parent_element.find_all(child_selector)
        return [element.get_text(strip=True) for element in child_elements]
    return []



In [40]:
# Extract the required fields
pmg_date = safe_extract(pmg_soup, 'h4', {}, default=np.nan)  # Extracts the date and NW552 reference
pmg_credentials = safe_extract(pmg_soup, 'h4', {'class': 'media-heading'}, default=np.nan)  # Extract the name and credentials of the person asking
pm_question = safe_extract(pmg_soup, 'p', {}, default=np.nan)  # Extract the question text

# Extract the content from all <p> tags within the specific 'committee-question-reply' div
pmg_answer_paragraphs = safe_extract_multiple(pmg_soup, 'div', {'class': 'committee-question-reply'})

# Join paragraphs into one string for better readability
pmg_answer = " ".join(pmg_answer_paragraphs)



# Output the extracted values
print("Date:", pmg_date)
print("Credentials:", pmg_credentials)
print("Question:", pm_question)
print("Answer:", pmg_answer)

Date: 04 October 2024 -NW1024
Credentials: Tetyana, Mr Yto ask the
Minister of Tourism
Question: What (a) urgent steps have been taken to revive (i) rural and (ii) township tourism in the Republic and (b) key (i) strategies and (ii) initiatives are in place that will be implemented to support and promote tourism in the specified areas?
Answer: (a)(i) and (ii) What urgent steps have been taken to revive (i) rural tourism and (ii) and township tourism in the Republic (b) What key (i) strategies are in place that will be implemented to support and promote tourism in the specified areas? The strategy involves increasing geographic spread of visitors, the diversification of products and inclusive participation by communities. (b) What (ii) initiatives are in place that will be implemented to support and promote tourism in the specified areas? I am further informed by South African Tourism that In support of the rural and township areas, they have implemented the following programmes: The Na